In [50]:
with open("input.txt") as fh:
    data = []
    item = {}
    for line in fh.readlines():
        line = line.strip()
        if line == "":
            data.append(item)
            item = {}
            continue
        
        values = line.split(" ")
        for value in values:
            i, j = value.split(":")
            item[i] = j #[ord(char) for char in j]
    data.append(item)

## Part 1

In [51]:
import cudf
import pandas as pd

In [52]:
df = pd.DataFrame(data)
df.head()

,hcl,ecl,iyr,pid,hgt,eyr,byr,cid
0,#6b5442,brn,2019,637485594,171cm,2021,1986,NaN
1,#341e13,NaN,1938,70195175,66cm,2025,2014,NaN
2,#efcc98,hzl,2011,589700330,174cm,2020,NaN,NaN
3,#bba027,brn,2028,153cm,173cm,2027,2004,54
4,b45cec,oth,2011,178cm,185cm,2029,NaN,NaN


In [53]:
gdf = cudf.DataFrame.from_pandas(df)
gdf.head()

,hcl,ecl,iyr,pid,hgt,eyr,byr,cid
0,#6b5442,brn,2019,637485594,171cm,2021,1986,<NA>
1,#341e13,<NA>,1938,70195175,66cm,2025,2014,<NA>
2,#efcc98,hzl,2011,589700330,174cm,2020,<NA>,<NA>
3,#bba027,brn,2028,153cm,173cm,2027,2004,54
4,b45cec,oth,2011,178cm,185cm,2029,<NA>,<NA>


In [54]:
one_nan = gdf[gdf.isnull().sum(axis=1) < 2]
one_nan

,hcl,ecl,iyr,pid,hgt,eyr,byr,cid
0,#6b5442,brn,2019,637485594,171cm,2021,1986,<NA>
3,#bba027,brn,2028,153cm,173cm,2027,2004,54
6,6962f7,oth,1974,2616015,191cm,2025,2015,<NA>
7,#18171d,grn,2019,268398556,67cm,2027,1951,<NA>
8,#623a2f,brn,1957,183179186,153cm,2029,2013,<NA>
...,...,...,...,...,...,...,...,...
252,3f59a6,oth,2018,217404607,179cm,2036,2025,<NA>
253,#efcc98,amb,2012,820370865,170cm,2025,1967,309
255,#efcc98,brn,2014,556900517,186cm,<NA>,1971,334
257,a3046e,lzr,2029,626863952,193cm,2024,2000,233


In [55]:
del gdf["cid"]
gdf

,hcl,ecl,iyr,pid,hgt,eyr,byr
0,#6b5442,brn,2019,637485594,171cm,2021,1986
1,#341e13,<NA>,1938,70195175,66cm,2025,2014
2,#efcc98,hzl,2011,589700330,174cm,2020,<NA>
3,#bba027,brn,2028,153cm,173cm,2027,2004
4,b45cec,oth,2011,178cm,185cm,2029,<NA>
...,...,...,...,...,...,...,...
254,#602927,gry,<NA>,008495978,159cm,2024,1940
255,#efcc98,brn,2014,556900517,186cm,<NA>,1971
256,#7d3b0c,grn,2020,<NA>,193cm,<NA>,1928
257,a3046e,lzr,2029,626863952,193cm,2024,2000


In [101]:
valid_passports = gdf[~gdf.isna().any(axis=1)]
valid_passports

,hcl,ecl,iyr,pid,hgt,eyr,byr
0,#6b5442,brn,2019,637485594,171cm,2021,1986
3,#bba027,brn,2028,153cm,173cm,2027,2004
6,6962f7,oth,1974,2616015,191cm,2025,2015
7,#18171d,grn,2019,268398556,67cm,2027,1951
8,#623a2f,brn,1957,183179186,153cm,2029,2013
...,...,...,...,...,...,...,...
251,#cfa07d,brn,2010,379769214,169cm,2027,1960
252,3f59a6,oth,2018,217404607,179cm,2036,2025
253,#efcc98,amb,2012,820370865,170cm,2025,1967
257,a3046e,lzr,2029,626863952,193cm,2024,2000


## Part 2

I'm using regular expressions which makes me sad.

### Hair

In [102]:
valid_passports = valid_passports[valid_passports.hcl.str.match("^#[0-9a-f]{6}$")]
len(valid_passports)

146

### Eye colour

In [103]:
valid_passports = valid_passports[valid_passports.ecl.str.match("^(amb|blu|brn|gry|grn|hzl|oth)$")]
len(valid_passports)

134

### Passport ID

In [104]:
valid_passports = valid_passports[valid_passports.pid.str.match("^[0-9]{9}$")]
len(valid_passports)

127

### Birth year

In [111]:
valid_passports = valid_passports[valid_passports.byr.str.match("^(19|20)(00|01|02|[2-9]{1}[0-9]{1})$")]
len(valid_passports)

125

### Expiration year

In [113]:
valid_passports = valid_passports[valid_passports.eyr.str.match("^20(20|21|22|23|24|25|26|27|28|29|30)$")]
len(valid_passports)

120

### Issue year

In [114]:
valid_passports = valid_passports[valid_passports.iyr.str.match("^20(10|11|12|13|14|15|16|17|18|19|20)$")]
len(valid_passports)

118

### Height

In [134]:
valid_passports = valid_passports[valid_passports.hgt.str.match("^1(9[0-3]|[5-8][0-9])cm$") | valid_passports.hgt.str.match("^(59|6[0-9]|7[0-6])in$")]
len(valid_passports)

114